In [30]:
from model import PoseFlowModel
from train import Trainer

In [27]:
model = PoseFlowModel(block_size=50, pose_embd=228, block_embd=176, num_layers=12, num_heads=8, dropout=0.2)

In [31]:
trainer = Trainer(model)